<a href="https://colab.research.google.com/github/krishnayaswanthp-netizen/Bank-management-system/blob/main/Innomatics_Research_labs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3
import json

Loading the orders dataset into a variable 'orders'


In [10]:
orders = pd.read_csv('orders.csv')
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


Loading users data onto a variable 'users'

In [11]:
with open('users.json', 'r') as f:
    users_data = json.load(f)
users = pd.DataFrame(users_data)
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


creating a temparary sqlite connection

In [5]:
conn = sqlite3.connect(':memory:')

Loading restaurant data on to the variable 'restaurants'

In [12]:
with open('restaurants.sql', 'r') as f:
    conn.executescript(f.read())
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)


OperationalError: table restaurants already exists

In [13]:
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


merging the orders and users data

In [7]:
merged_df = pd.merge(orders, users, on='user_id', how='left')

merging the restaurant and the (orders , users) datasets on to a variable called "final_df"(where i can finally work on)

In [8]:
final_df = pd.merge(merged_df, restaurants, on='restaurant_id', how='left')

In [9]:
final_df.to_csv('final_food_delivery_dataset.csv', index=False)

In [14]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


i have finally merged the orders with user dataset , and then that dataset with restaurants and got "dinal_df" as my dataset where i can work on

Which city has the highest total revenue (total_amount) from Gold members?  

Q1 (Gold Revenue by City):



In [16]:
final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().idxmax()

'Chennai'

Which cuisine has the highest average order value across all orders?
Q2 (Avg Order Value by Cuisine):

In [17]:
final_df.groupby('cuisine')['total_amount'].mean().idxmax()

'Mexican'

Q3: How many distinct users placed orders worth more than ₹1000 in total?

In [18]:
user_totals = final_df.groupby('user_id')['total_amount'].sum()
distinct_users_count = (user_totals > 1000).sum()
distinct_users_count

np.int64(2544)

Q4: Which restaurant rating range generated the highest total revenue?

In [19]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels, include_lowest=True)

rating_revenue = final_df.groupby('rating_range')['total_amount'].sum()

/tmp/ipython-input-905955335.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_revenue = final_df.groupby('rating_range')['total_amount'].sum()


In [20]:
rating_revenue

,total_amount
rating_range,
3.0 – 3.5,2136772.70
3.6 – 4.0,1717494.41
4.1 – 4.5,1960326.26
4.6 – 5.0,2197030.75


Q5: Among Gold members, which city has the highest average order value?

In [23]:
gold_city_aov = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].mean()
gold_city_aov.idxmax()

'Chennai'

Q6: Cuisine with lowest distinct restaurants but significant revenue?

In [24]:
cuisine_stats = final_df.groupby('cuisine').agg({'restaurant_id': 'nunique', 'total_amount': 'sum'})
print(cuisine_stats.sort_values(by='restaurant_id'))

         restaurant_id  total_amount
cuisine                             
Chinese            120    1930504.65
Indian             126    1971412.58
Italian            126    2024203.80
Mexican            128    2085503.09


hence the answer is chinese


Q7: What percentage of total orders were placed by Gold members?

In [25]:
gold_percentage = (len(final_df[final_df['membership'] == 'Gold']) / len(final_df)) * 100
r=round(gold_percentage)
r

50

Q8: Highest average order value but less than 20 total orders?

In [26]:
res_analysis = final_df.groupby('restaurant_name_x').agg({'total_amount': 'mean', 'order_id': 'count'})
eligible_res = res_analysis[res_analysis['order_id'] < 20]
eligible_res['total_amount'].idxmax()

'Hotel Dhaba Multicuisine'

Q9: Which combination contributes the highest revenue?

In [27]:
# Grouping by both Membership and Cuisine
best_combo = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum()
best_combo.idxmax()

('Regular', 'Mexican')

Q10: During which quarter of the year is the total revenue highest?

In [28]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)
final_df['quarter'] = final_df['order_date'].dt.quarter

quarter_rev = final_df.groupby('quarter')['total_amount'].sum()
quarter_rev.idxmax()

np.int32(3)

section 2:

Q1

In [29]:
gold_orders_count = len(final_df[final_df['membership'] == 'Gold'])
print(f"Total Gold orders: {gold_orders_count}")

Total Gold orders: 4987


Q2

In [31]:
total_revenue_hyd=final_df[final_df['city']=='Hyderabad']['total_amount'].sum()
round(total_revenue_hyd)

1889367

Q3

In [32]:
distinct_users = final_df['user_id'].nunique()
distinct_users

2883

Q4

In [36]:
gold_aov = final_df[final_df['membership'] == 'Gold']['total_amount'].mean()
round(gold_aov,2)

np.float64(797.15)

5Q

In [40]:
high_rated_orders = len(final_df[final_df['rating'] >= 4.5])
high_rated_orders

3374

6Q

In [43]:
gold_data = final_df[final_df['membership'] == 'Gold']
top_city = gold_data.groupby('city')['total_amount'].sum().idxmax()

final_answer = len(gold_data[gold_data['city'] == top_city])
final_answer

1337

In [42]:
top_gold_city

'Chennai'

In [44]:
len(final_df)

10000

In [45]:
missing_users = final_df[final_df['membership'].isna()]

if missing_users.empty:
    print("Every order has a matching user record.")
else:
    print(f"Found {len(missing_users)} orders with no matching user data.")
    print(missing_users.head())

Every order has a matching user record.
